# Resources
* [jupyter.org/widgets](https://jupyter.org/widgets)
* [ipywidgets `interact`](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html)
* [TowardDataScience.com blog](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)

In [1]:
import ipywidgets as wiggy
from IPython.display import display
from crowd_truth_amt import CrowdTruth
import random

# Guess The Theme

1. pick a relation/theme at random (omit `Other` and `NoMeaning`)
2. find all responses from that theme
3. read the first 20 responses
4. guess the theme

In [2]:
ai_afraid_ad_own_words_1_path = '../crowd_truth/analyst_institute_afraid_ad_message_testing_saying_in_your_own_words_1.csv'
ct = CrowdTruth(df=ai_afraid_ad_own_words_1_path)

In [3]:
def get_theme_and_responses():
    themes = list(ct.relations)
    guess_theme = random.choice(themes)
    guess_theme_inds = ct.clarity_df[ct.clarity_df[guess_theme] > 1].index
    responses = list(ct.responses.loc[guess_theme_inds])
    random.shuffle(responses)
    
    return guess_theme, responses

# Full Layout

In [4]:
theme, responses = get_theme_and_responses()
num_responses=5
guesses = []
dropdown_guesses = []
guess_input = wiggy.Text(value='Put your best guess here.')
submit_btn = wiggy.Button(description='Submit')
five_more_responses_btn = wiggy.Button(description='See 5 more responses')

right_box = wiggy.VBox([guess_input, submit_btn, five_more_responses_btn])

html_responses = wiggy.HTML(value=' <br> '.join(responses[:num_responses]))

def more_responses_handler(obj):
    global num_responses
    num_responses += 5
    html_responses.value=' <br> '.join(responses[:num_responses])

five_more_responses_btn.on_click(more_responses_handler)


def submit_handler(obj):
    guesses.append(guess_input.value)
    global box
    box.close()
    drop = wiggy.Dropdown(
        options=ct.relations,
        value=random.choice(ct.relations),
        description='Choose a theme',
        disabled=False,
    )
    
    html_responses = wiggy.HTML(value=' <br> '.join(responses[:num_responses]))
    right_box = wiggy.VBox([drop])
    box = wiggy.HBox([html_responses, right_box])
    display(box)
    drop.observe(dropdown_on_change)

def dropdown_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        dropdown_guesses = change['new']
        box.close()
        if change['new'] == theme:
            print("Correct! %s was the right theme" % theme)
        else:
            print("Thank you for your guess. %s was the theme determined by our featurization and turk annotation process")
        
submit_btn.on_click(submit_handler)
box = wiggy.HBox([html_responses, right_box])
display(box)